# Importing libraries

In [1]:
import sys
import os
import json
import numpy as np
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import normalize_new_row_with_mean_std
import asyncio
import threading
from websocket_client import fetch_market_data, get_latest_ohlc_volume, market_data_store, get_market_summary
import time
from utils_data import TechnicalIndicatorWindow
from utils_data import  UpstoxTrader
import pytz
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

# Getting Real Time Data

In [ ]:
# Run the WebSocket client in a background thread
async def start_connection():
    while True:
        try:
            await fetch_market_data(symbol="SUZLON")
        except Exception as e:
            print(f"Error: {e}, reconnecting in 5 seconds...")
            await asyncio.sleep(5)

def start_ws_loop():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_connection())

# Start thread
thread = threading.Thread(target=start_ws_loop)
thread.start()
# Give it time to connect and start receiving
time.sleep(5)


{'status': 'success', 'data': {'authorizedRedirectUri': 'wss://wsfeeder-api.upstox.com/market-data-feeder/v3/upstox-developer-api/feeds?requestId=f23f5ba2-f575-475d-9a8f-d12a1275a8c9&code=nQNPZ-db3dfeb9-a7b7-4efc-be86-6b022ab19fcf', 'authorized_redirect_uri': 'wss://wsfeeder-api.upstox.com/market-data-feeder/v3/upstox-developer-api/feeds?requestId=f23f5ba2-f575-475d-9a8f-d12a1275a8c9&code=nQNPZ-db3dfeb9-a7b7-4efc-be86-6b022ab19fcf'}}
Connection established


Exception in thread Thread-3 (start_ws_loop):
Traceback (most recent call last):
  File "c:\Users\srija\anaconda3\envs\trader\Lib\site-packages\websockets\legacy\protocol.py", line 968, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\srija\anaconda3\envs\trader\Lib\site-packages\websockets\legacy\protocol.py", line 1038, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\srija\anaconda3\envs\trader\Lib\site-packages\websockets\legacy\protocol.py", line 1113, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\srija\anaconda3\envs\trader\Lib\site-packages\websockets\legacy\protocol.py", line 1170, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "c:\Users\srija\anaconda3\envs\trader\Lib\site-packages\websockets\le

In [4]:
latest = get_latest_ohlc_volume("SUZLON")
latest

{'timestamp': datetime.datetime(2025, 6, 18, 9, 15),
 'open': 64.52,
 'high': 64.68,
 'low': 63.85,
 'close': 64.22,
 'volume': '1379339'}

In [5]:
# Import Norm Param for converting real time data to normalized data
with open("C:/Users/srija/Assignment/Trading/json_files/suzlon_14_june_norm_params.json", "r") as f:
    norm_params = json.load(f)


In [6]:
def get_state(row):
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state

# Loading Models

In [7]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy_net.to(device)
target_net.to(device)

model_path = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june\suzlon_14_june_7661.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

# Action time

In [8]:
trader = UpstoxTrader()
symbol ="SUZLON"
ti_window = TechnicalIndicatorWindow(window_size=28)

In [ ]:
india = pytz.timezone('Asia/Kolkata')
now_ist = datetime.now(india)
market_close_time = now_ist.replace(hour=15, minute=10, second=0, microsecond=0)
while True:
    t1 = time.time()
    now_ist = datetime.now(india)
    market_status = trader.MarketStatus()
    if market_status is False or now_ist >= market_close_time:
            print("Market is closed.")
            break

    new_bar = get_latest_ohlc_volume(symbol=symbol)
    ti_window.update(new_bar)

    with open('C:/Users/srija/Assignment/Trading/json_files/today_suzlon.csv', 'a') as f:
        f.write(f"{new_bar['timestamp']},{new_bar['open']},{new_bar['high']},{new_bar['low']},{new_bar['close']},{new_bar['volume']}\n")

    features = ti_window.get_feature_row()
    if features is None:
        print("No features available yet.")
        time.sleep(60)
        continue
    features = features.rename({'timestamp': 'time'})
    normalized_row = normalize_new_row_with_mean_std(features, norm_params)
    state = get_state(normalized_row)
    t2 = time.time()
    print(f"Time taken to get state: {t2 - t1:.2f} seconds")
    with torch.no_grad():
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        q_values = policy_net(state)
        action=q_values.argmax().item()
    t3 =time.time()
    print(f"Time taken for inference: {t3 - t2:.2f} seconds")
    
    order_id = {}
    with ThreadPoolExecutor(max_workers=2) as executor:
        future_available_cash = executor.submit(trader.get_margin)
        future_latest_price = executor.submit(get_market_summary, symbol)
        available_cash = future_available_cash.result()
        latest_price = future_latest_price.result()['latest_price']
    
    quantity = int(available_cash*0.9*2.5/latest_price) # Use 90% of available cash for trading and 4x leverage
    if quantity <= 10:
        print("Insufficient funds to place an order.")
        time.sleep(60)
        continue
    t4 = time.time()
    print(f"Time taken to prepare order: {t4 - t3:.2f} seconds")
    try:
        if action == 0:
            print("No action taken.")

        elif action == 1:
            print("Placing buy order.")
            with ThreadPoolExecutor(max_workers=3) as executor:
                future_market = executor.submit(trader.IntraMarketOrder, symbol, quantity, "BUY")
                future_target = executor.submit(trader.IntraLimitOrder, symbol, quantity, "SELL", latest_price * 1.005)
                future_stop_loss = executor.submit(trader.IntraDayStopLossOrder, symbol, quantity, "SELL", latest_price * 0.99)

                market_order = future_market.result()
                target_order = future_target.result()
                stop_loss_order = future_stop_loss.result()

            order_id['market_order'] = market_order['data']['order_id']
            order_id['target_order'] = target_order['data']['order_id']
            order_id['stop_loss_order'] = stop_loss_order['data']['order_id']

        elif action == 2:
            print("Placing sell order.")
            with ThreadPoolExecutor(max_workers=3) as executor:
                future_market = executor.submit(trader.IntraMarketOrder, symbol, quantity, "SELL")
                future_target = executor.submit(trader.IntraLimitOrder, symbol, quantity, "BUY", latest_price * 0.995)
                future_stop_loss = executor.submit(trader.IntraDayStopLossOrder, symbol, quantity, "BUY", latest_price * 1.01)

                market_order = future_market.result()
                target_order = future_target.result()
                stop_loss_order = future_stop_loss.result()

            order_id['market_order'] = market_order['data']['order_id']
            order_id['target_order'] = target_order['data']['order_id']
            order_id['stop_loss_order'] = stop_loss_order['data']['order_id']
        t5 = time.time()
        print(f"Time taken to place orders: {t5 - t4:.2f} seconds")
        if action in [1, 2]:
            while True:
                with ThreadPoolExecutor(max_workers=2) as executor:
                    future_stat1 = executor.submit(trader.getOrderDetails, order_id['target_order'])
                    future_stat2 = executor.submit(trader.getOrderDetails, order_id['stop_loss_order'])
                    stat1 = future_stat1.result()
                    stat2 = future_stat2.result()
                if stat1['data']['status'] =='complete':
                    trader.CancelOrder(order_id['stop_loss_order'])
                    print("Target order executed, stop loss order cancelled.")
                    break
                elif stat2['data']['status'] =='complete':
                    trader.CancelOrder(order_id['target_order'])
                    print("Stop loss order executed, target order cancelled.")
                    break
                time.sleep(5)  # Check every 10 seconds
    except Exception as e:
        print(f"An error occurred while placing orders: {e}")
        continue

    # Wait for the next minute to start the next iteration
    next_minute = (datetime.now(india) + timedelta(minutes=1)).replace(second=0, microsecond=0)
    while datetime.now(india) < next_minute:
        time.sleep(0.5)
    time.sleep(3)  


No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
No features available yet.
Time taken to get state: 0.28 seconds
Time taken for inference: 0.07 seconds
Time taken to prepare order: 0.29 seconds
No action taken.
Time taken to place orders: 0.00 seconds
Time taken to get state: 0.28 seconds
Time taken for inference: 0.00 seconds
Time taken to pre

KeyboardInterrupt: 